# Setup code for enabling partitioning and replication

Append to redundancy.yml appropriate services 

Variables from original mongoDB config

In [62]:
#Parking manager config data
mongo_service_name_managerDB= 'mongo_manager_db_service' 
mongo_db_name_managerDB = 'ParkMan_manager_db'
mongo_user_managerDB = 'user_Park_Manager'
mongo_pass_managerDB = 'mongo_pass'  
mongo_port_managerDB = '27016'
mongo_express_port_managerDB = '8081'
mongo_collections_managerDB = '["parking_lots", "parking_spots", "owners"]'


###Shared vals
mongo_admin_user = 'admin'
mongo_admin_pass = 'admin_pass'

mongo_user_external="external" #user which can only read data from another mongoDB
mongo_user_external_pass="external_pass"


mongo_express_user = 'mongo_express_user'
mongo_express_pass = 'mongo_express_pass'


###

#UserDB config data
mongo_service_name_userDB= 'mongo_user_db_service' 
mongo_db_name_userDB = 'ParkMan_user_db'
mongo_user_userDB = 'user_Person'
mongo_pass_userDB = 'mongo_pass'  
mongo_port_userDB = '27017'
mongo_express_port_userDB = '8082'
mongo_collections_userDB = '["users"]'

Create dockerfile for router

In [145]:
%%writefile ./ManagerDB.Router.Dockerfile

FROM mongo:latest

RUN mkdir /scripts

COPY ./Partitioning_replication/ManagerMongoDB/sharded-mongoDB-init.sh /scripts

RUN chmod +x /scripts/sharded-mongoDB-init.sh

Overwriting ./ManagerDB.Router.Dockerfile


Create modified MongoDB init script

In [141]:
%%writefile ./sharded-mongoDB-init.sh


#!/bin/bash
set -e

# Ensure necessary environment variables are set
: "${MONGO_DATABASE_NAME:?Environment variable MONGO_DATABASE_NAME not set}"
: "${MONGO_DATABASE_USER:?Environment variable MONGO_DATABASE_USER not set}"
: "${MONGO_DATABASE_PASS:?Environment variable MONGO_DATABASE_PASS not set}"
: "${MONGO_INITDB_ROOT_USERNAME:?Environment variable MONGO_INITDB_ROOT_USERNAME not set}"
: "${MONGO_INITDB_ROOT_PASSWORD:?Environment variable MONGO_INITDB_ROOT_PASSWORD not set}"
: "${MONGO_COLLECTION_NAMES:?Environment variable MONGO_COLLECTION_NAMES not set}"
: "${MONGO_DATABASE_EXTERNAL_USER:?Environment variable MONGO_DATABASE_EXTERNAL_USER not set}"
: "${MONGO_DATABASE_EXTERNAL_PASS:?Environment variable MONGO_DATABASE_EXTERNAL_PASS not set}"

echo "Starting MongoDB initialization script..."


# Switch to admin database to create the app user
mongosh admin <<EOF
var user = db.getSiblingDB('$MONGO_DATABASE_NAME').getUser('$MONGO_DATABASE_USER');
if (!user) {
    // Create application database and user
    db.getSiblingDB('$MONGO_DATABASE_NAME').createUser({
      user: "$MONGO_DATABASE_USER",
      pwd: "$MONGO_DATABASE_PASS",
      roles: [
        { role: "readWrite", db: "$MONGO_DATABASE_NAME" }
      ]
    });
    print("Application user '$MONGO_DATABASE_USER' created successfully on '$MONGO_DATABASE_NAME'.");
} else {
    print("User '$MONGO_DATABASE_USER' already exists on '$MONGO_DATABASE_NAME'.");
}
#create user which can only read DB
var external_user = db.getSiblingDB('$MONGO_DATABASE_NAME').getUser('$MONGO_DATABASE_EXTERNAL_USER');
if (!external_user) {
    // Create application database and user
    db.getSiblingDB('$MONGO_DATABASE_NAME').createUser({
      user: "$MONGO_DATABASE_EXTERNAL_USER",
      pwd: "$MONGO_DATABASE_EXTERNAL_PASS",
      roles: [
        { role: "read", db: "$MONGO_DATABASE_NAME" }
      ]
    });
    print("Application user '$MONGO_DATABASE_EXTERNAL_USER' created successfully on '$MONGO_DATABASE_NAME'.");
} else {
    print("User '$MONGO_DATABASE_EXTERNAL_USER' already exists on '$MONGO_DATABASE_NAME'.");
}
EOF

# Initialize collections if they don't exist
mongosh "$MONGO_DATABASE_NAME" -u "$MONGO_DATABASE_USER" -p "$MONGO_DATABASE_PASS" --authenticationDatabase "$MONGO_DATABASE_NAME" <<EOF
var collections = $MONGO_COLLECTION_NAMES;
collections.forEach(function(collection) {
  if (db.getCollectionNames().indexOf(collection) === -1) {
  db.createCollection(collection);
  print("Collection '" + collection + "' created successfully in database '$MONGO_DATABASE_NAME'.");
  } else {
  print("Collection '" + collection + "' already exists in database '$MONGO_DATABASE_NAME'.");
  }
});
EOF

echo "MongoDB initialization complete."





echo "Begin sharding config for managerDB"

mongosh admin<< EOF
sh.enableSharding('$MONGO_DATABASE_NAME');
var db = db.getSiblingDB(dbName);
db.parking_lots.createIndex({ geolocation: "2dsphere" });
sh.shardCollection('{$MONGO_DATABASE_NAME}.parking_lots', { geolocation: "hashed" });
EOF

echo "MongoDB mongos sharding setup complete."




Overwriting ./sharded-mongoDB-init.sh


Create parititoned and replicated ManagerDB database

In [132]:
from ruamel.yaml import YAML

# Initialize YAML parser
yaml = YAML()
yaml.preserve_quotes = True  # Preserves quotes in the YAML file
yaml.indent(mapping = 2, sequence = 2, offset = 2)

#Setup file edit path 
docker_compose_path = '../../redundancy-manager-db.yml'  

# Read the docker-compose.yml file
with open(docker_compose_path, 'r') as file:
    docker_compose = yaml.load(file)


services={

"configsvr1": {
            'image': 'mongo:latest',
            'hostname': 'configsvr1',
            'command': ["mongod", "--replSet", "configReplSet", "--configsvr", "--port", "27019", "--dbpath" , "/data/db", "--bind_ip_all"],
            'ports': ['27019:27019'],
            'volumes': ['configsvr1_data:/data/db', ],
            # 'healthcheck': {'test': ["CMD", "mongo", "--eval", "db.adminCommand('ping')"],
            #                  'interval': '30s', 
            #                  'timeout': '10s',
            #                    'retries': '3' ,
            #                    'start_period': '10s'},
            'networks': ['app-network'],
            'restart':'always',
            'links':["shard1_1","shard2_1"],
            
            
        },
            

"configsvr2": {
            'image': 'mongo:latest',
            'hostname': 'configsvr2',
            'command': ["mongod", "--replSet", "configReplSet", "--configsvr", "--port", "27019", "--dbpath" , "/data/db", "--bind_ip_all"],
            'ports': ['27029:27019'],
            'volumes': ['configsvr2_data:/data/db'],
            # 'healthcheck': {'test': ["CMD", "mongo", "--eval", "db.adminCommand('ping')"],
            #                  'interval': '30s', 
            #                  'timeout': '10s',
            #                    'retries': '3' ,
            #                    'start_period': '10s'},
            'networks': ['app-network'],
            'restart':'always',
            'links':["configsvr1"]
        },


"configsvr3" : {
            'image': 'mongo:latest',
            'hostname': 'configsvr3',
            'command': ["mongod", "--replSet", "configReplSet", "--configsvr", "--port", "27019", "--dbpath" , "/data/db", "--bind_ip_all"],
            'ports': ['27039:27019'],
            'volumes': ['configsvr3_data:/data/db'],
            # 'healthcheck': {'test': ["CMD", "mongo", "--eval", "db.adminCommand('ping')"],
            #                  'interval': '30s', 
            #                  'timeout': '10s',
            #                    'retries': '3' ,
            #                    'start_period': '10s'},
            'networks': ['app-network'],
            'restart':'always',
            'links':["configsvr2"]
        },

"shard1_1": {
            'image': 'mongo:latest',
            'hostname':'shard1_1',
            'command': ['mongod', '--replSet', 'shard1ReplSet', '--shardsvr', '--port', '27018', "--dbpath" , "/data/db", "--bind_ip_all"],
            'ports': ['27018:27018'],
            'volumes': ['shard1_1_data:/data/db',],
            # 'healthcheck': {'test': ["CMD", "mongo", "--eval", "db.adminCommand('ping')"],
            #                  'interval': '30s', 
            #                  'timeout': '10s',
            #                    'retries': '3' ,
            #                    'start_period': '10s'},
            'networks': ['app-network'],
            'restart':'always',
            'links':["shard1_2","shard1_3"]
        },

"shard1_2":{
            'image': 'mongo:latest',
            'hostname':'shard1_2',
            'command': ['mongod', '--replSet', 'shard1ReplSet', '--shardsvr', '--port', '27018',"--dbpath" , "/data/db", "--bind_ip_all"],
            'ports': ['27028:27018'],
            'volumes': ['shard1_2_data:/data/db',],
            # 'healthcheck': {'test': ["CMD", "mongo", "--eval", "db.adminCommand('ping')"],
            #                  'interval': '30s', 
            #                  'timeout': '10s',
            #                    'retries': '3' ,
            #                    'start_period': '10s'},
            'networks': ['app-network'],
            'restart':'always'
        },

"shard1_3": {
            'image': 'mongo:latest',
            'hostname':'shard1_3',
            'command': ['mongod', '--replSet', 'shard1ReplSet', '--shardsvr', '--port', '27018',"--dbpath" , "/data/db", "--bind_ip_all"],
            'ports': ['27038:27018'],
            'volumes': ['shard1_3_data:/data/db', ],
            # 'healthcheck': {'test': ["CMD", "mongo", "--eval", "db.adminCommand('ping')"],
            #                  'interval': '30s', 
            #                  'timeout': '10s',
            #                    'retries': '3' ,
            #                    'start_period': '10s'},
            'networks': ['app-network'],
            'restart':'always'
        },

"shard2_1": {
            'image': 'mongo:latest',
            'hostname':'shard2_1',
            'command': ['mongod', '--replSet', 'shard2ReplSet', '--shardsvr', '--port', '27018', "--dbpath" , "/data/db", "--bind_ip_all"],
            'ports': ['27048:27018'],
            'volumes': ['shard2_1_data:/data/db', ],
            # 'healthcheck': {'test': ["CMD", "mongo", "--eval", "db.adminCommand('ping')"],
            #                  'interval': '30s', 
            #                  'timeout': '10s',
            #                    'retries': '3' ,
            #                    'start_period': '10s'},
            'networks': ['app-network'],
            'restart':'always',
            'links':["shard2_2","shard2_3"]
        },

"shard2_2": {
            'image': 'mongo:latest',
            'hostname':'shard2_2',
            'command': ['mongod', '--replSet', 'shard2ReplSet', '--shardsvr', '--port', '27018', "--dbpath" , "/data/db", "--bind_ip_all"],
            'ports': ['27058:27018'],
            'volumes': ['shard2_2_data:/data/db', ],
            # 'healthcheck': {'test': ["CMD", "mongo", "--eval", "db.adminCommand('ping')"],
            #                  'interval': '30s', 
            #                  'timeout': '10s',
            #                    'retries': '3' ,
            #                    'start_period': '10s'},
            'networks': ['app-network'],
            'restart':'always'
        },

"shard2_3": {
            'image': 'mongo:latest',
            'hostname':'shard2_3',
            'command': ['mongod', '--replSet', 'shard2ReplSet', '--shardsvr', '--port', '27018',"--dbpath" , "/data/db", "--bind_ip_all"],
            'ports': ['27068:27018'],
            'volumes': ['shard2_3_data:/data/db', ],
            # 'healthcheck': {'test': ["CMD", "mongo", "--eval", "db.adminCommand('ping')"],
            #                  'interval': '30s', 
            #                  'timeout': '10s',
            #                    'retries': '3' ,
            #                    'start_period': '10s'},
            'networks': ['app-network'],
            'restart':'always'
        },

#Mongos Router
f'{mongo_service_name_managerDB}': {
            'build': {'context':'.', 'dockerfile':'./Partitioning_replication/ManagerMongoDB/ManagerDB.Router.Dockerfile'}, 
            'container_name': mongo_service_name_managerDB,
            'env_file' : [
                './.env',
                './mongoDB/.env.shared.express.ManagerDB',
                './mongoDB/.env.shared.ManagerApp',
                './mongoDB/.ManagerDB.env',
                './mongoDB/.ManagerDB.UserDB.env.shared'
            ],
            'command': ['mongos', '--configdb', 'configReplSet/configsvr1:27019,configsvr2:27019,configsvr3:27019', '--port', '27017', '--bind_ip_all'],
            'ports': ['${MONGO_PORT_MANAGERDB}:27017'],
            'restart':'always',
            
            'depends_on': [
                'configsvr1', 'configsvr2', 'configsvr3',
                'shard1_1', 'shard1_2', 'shard1_3',
                'shard2_1', 'shard2_2', 'shard2_3'
            ],
            'networks': ['app-network'],
            'healthcheck': {'test': ["CMD", "mongosh", "--eval", "db.adminCommand('ping')"],
                              'interval': '30s', 
                              'timeout': '10s',
                                'retries': '3' ,
                                'start_period': '10s'},
        }
}


if docker_compose["services"] is None:
    docker_compose["services"]=dict()

docker_compose["services"].update(services)

if docker_compose["volumes"] is None:
    docker_compose["volumes"]=dict()
docker_compose["volumes"].update({
        'configsvr1_data': {},
        'configsvr2_data': {},
        'configsvr3_data': {},
        'shard1_1_data': {},
        'shard1_2_data': {},
        'shard1_3_data': {},
        'shard2_1_data': {},
        'shard2_2_data': {},
        'shard2_3_data': {}
    })



# Write the updated configuration back to docker-compose.yml
with open(docker_compose_path, 'w') as file:
    yaml.dump(docker_compose, file)

print("\ndocker-compose.yml has been updated successfully.")








docker-compose.yml has been updated successfully.


## EXECUTE THESE CELLS TO INIT DB (Only during first setup)

Wait a bit for setup to finish and then setup router

In [1]:
# Init the replica sets (use the MONGOS host)
!docker exec -it mongo_manager_db_service bash -c "echo 'rs.initiate({_id: \"configReplSet\",configsvr: true, members: [{ _id : 0, host : \"configsvr1:27019\", priority: 2 },{ _id : 1, host : \"configsvr2:27019\" }, { _id : 2, host : \"configsvr3:27019\" }]})' | mongosh --host configsvr1:27019"
!docker exec -it mongo_manager_db_service bash -c "echo 'rs.initiate({_id : \"shard1ReplSet\", members: [{ _id : 0, host : \"shard1_1:27018\", priority: 2 },{ _id : 1, host : \"shard1_2:27018\" },{ _id : 2, host : \"shard1_3:27018\" }]})' | mongosh --host shard1_1:27018"
!docker exec -it mongo_manager_db_service bash -c "echo 'rs.initiate({_id : \"shard2ReplSet\", members: [{ _id : 0, host : \"shard2_1:27018\", priority: 2 },{ _id : 1, host : \"shard2_2:27018\" },{ _id : 2, host : \"shard2_3:27018\" }]})' | mongosh --host shard2_1:27018"

]0;mongosh mongodb://configsvr1:27019/?directConnection=trueCurrent Mongosh Log ID:	678d0fec997e794287e94969
Connecting to:		mongodb://configsvr1:27019/?directConnection=true&appName=mongosh+2.3.4
Using MongoDB:		8.0.4
Using Mongosh:		2.3.4

For mongosh info see: https://www.mongodb.com/docs/mongodb-shell/


To help improve our products, anonymous usage data is collected and sent to MongoDB periodically (https://www.mongodb.com/legal/privacy-policy).
You can opt-out by running the disableTelemetry() command.

 ------
   The server generated these startup warnings when booting
   2025-01-19T14:44:47.623+00:00: Using the XFS filesystem is strongly recommended with the WiredTiger storage engine. See http://dochub.mongodb.org/core/prodnotes-filesystem
   2025-01-19T14:44:47.830+00:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted
   2025-01-19T14:44:47.830+00:00: For customers running the current memory allocator, we sugge

Then wait until appropriate leaders are elected (about 5 seconds) before running this cell

In [5]:
# ADD TWO SHARDS (mongors1, and mongors2)
!docker exec -it mongo_manager_db_service bash -c "echo 'sh.addShard(\"shard1ReplSet/shard1_1:27018,shard1_2:27018,shard1_3:27018\")' | mongosh "
!docker exec -it mongo_manager_db_service bash -c "echo 'sh.addShard(\"shard2ReplSet/shard2_1:27018,shard2_2:27018,shard2_3:27018\")' | mongosh"

]0;mongosh mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000Current Mongosh Log ID:	678d0ffbf6433664d4e94969
Connecting to:		mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.3.4
Using MongoDB:		8.0.4
Using Mongosh:		2.3.4

For mongosh info see: https://www.mongodb.com/docs/mongodb-shell/

 ------
   The server generated these startup warnings when booting
   2025-01-19T14:44:50.475+00:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted
------

[direct: mongos] test> sh.addShard("shard1ReplSet/shard1_1:27018,shard1_2:27018[direct: mongos] test> sh.addShard("shard1ReplSet/shard1_1:27018,shard1_2:27018, shard1_3:27018")
{
  shardAdded: 'shard1ReplSet',
  ok: 1,
  '$clusterTime': {
    clusterTime: Timestamp({ t: 1737297916, i: 20 }),
    signature: {
      hash: Binary.createFromBase64('AAAAAAAAAAAAAAAAAAAAAAAAAAA=', 0),
      keyId: Long('0')
    

Finally execute mongo-init.sh script and sharding on DB

In [6]:
!docker exec -it mongo_manager_db_service bash -c "./scripts/sharded-mongoDB-init.sh"

Starting MongoDB initialization script...
]0;mongosh mongodb://127.0.0.1:27017/admin?directConnection=true&serverSelectionTimeoutMS=2000Current Mongosh Log ID:	678d10049c062fcc64e94969
Connecting to:		mongodb://127.0.0.1:27017/admin?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.3.4
Using MongoDB:		8.0.4
Using Mongosh:		2.3.4

For mongosh info see: https://www.mongodb.com/docs/mongodb-shell/

 ------
   The server generated these startup warnings when booting
   2025-01-19T14:44:50.475+00:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted
------

[direct: mongos] admin> var user = db.getSiblingDB('ParkMan_manager_db').getUse[direct: mongos] admin> var user = db.getSiblingDB('ParkMan_manager_db').getUser ('user_Park_Manager');

[direct: mongos] admin> if (!user) {
...     // Create application database and user
...     db.getSiblingDB('ParkMan_manager_db').createUser({
...       user: "user_Park_Ma